# RedditWebScraping

TODO :

-- Problem Statement : Scraping the tops communities in all categories on Reddit web page

-- Tools Used : Selenium, Beautiful Soup, Python, Pandas

-- Request you to go through the readme file for more clarification


### Importing the Required Libraries

In [25]:
import requests

In [26]:
home_url="https://www.reddit.com"

In [27]:
response=requests.get(home_url)

In [28]:
response.status_code

200

In [29]:
page_contents=response.text

In [68]:
with open('redditpage.html','w',encoding='utf-8') as f:
    f.write(page_contents)

### Above request library is not fetching the entire html page, so using selenium web driver.

In [1]:
import pandas as pd

In [2]:
from bs4 import BeautifulSoup

In [3]:
from selenium import webdriver

In [4]:
 driver = webdriver.Chrome()

In [5]:
home_url="https://www.reddit.com"

In [6]:
driver.get(home_url)

In [7]:
contents = BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
with open('redditpage.html','w',encoding='utf-8') as f:
    f.write(str(contents))

### Function Declarations

In [8]:
def get_inner_div_tags(outer_div_tags):
    inner_tags=[]
    for outer_div_tag in outer_div_tags:
        inner_tag=outer_div_tag.find_all('div',{'class':'_2mHuuvyV9doV3zwbZPtIPG'})
        #inner_tags.append(inner_tag)
        inner_tags=inner_tags+inner_tag
    return inner_tags

In [9]:
def get_a_tags(inner_div_tag):
    a_tags=[]
    for div_tag in inner_div_tag:
        a_tag=div_tag.find_all('a',{'class':'_13rO9T2auFedwF7O7V72O6'})
        #a_tags.append(a_tag)
        a_tags=a_tags+a_tag
    return a_tags

In [10]:
def get_topic_details(a_tags):
    names=[]
    urls=[]
    for topic_detail in a_tags:
        names.append(topic_detail['title'])
        urls.append(home_url+topic_detail['href'])
    return names,urls

In [11]:
def get_topic_infos(urls):
    desc=[]
    mem=[]
    online=[]
    date=[]
    for url in urls:
        driver.get(url)
        content = BeautifulSoup(driver.page_source, 'html.parser')
        info_tag=content.find('div',{'class':'TmgZY6tDcdErbE5d7E0HJ'})
        topicdesc,topicmem,topiconline,topicdate=get_topic_info(info_tag)
        desc.append(topicdesc)
        mem.append(topicmem)
        online.append(topiconline)
        date.append(topicdate[8:len(topicdate)])
    return desc,mem,online,date

In [20]:
def get_topic_info(info_tag):
    if(info_tag):
        des=info_tag.find('div',{'class':'_1zPvgKHteTOub9dKkvrOl4'})
        if(des):
            topic_desc=des.text
        else:
            topic_desc=''
        #Getting the followers
        members=info_tag.find('div',{'class':'nEdqRRzLEN43xauwtgTmj'})
        if(members):
            mem=members.find('div',{'class':'_3XFx6CfPlg-4Usgxm0gK8R'})
            if(mem):
                topic_mem=mem.text
            else:
                topic_mem=''
        else:
            topic_mem=''
        #Getting the online
        onlines=info_tag.find('div',{'class':'_3_HlHJ56dAfStT19Jgl1bF'})
        if(onlines):
            onl=onlines.find('div',{'class':'_3XFx6CfPlg-4Usgxm0gK8R'})
            if(onl):
                topic_onl=onl.text
            else:
                topic_onl=''
        else:
            topic_onl=''
        #Getting the creation date
        date=info_tag.find('div',{'class':'_2QZ7T4uAFMs_N83BZcN-Em'})
        if(date):
            topic_date=date.text
        else:
            topic_date=''
    else:
        topic_desc=''
        topic_mem=''
        topic_onl=''
        topic_date=''
        return topic_desc,topic_mem,topic_onl,topic_date
    return topic_desc,topic_mem,topic_onl,topic_date

### Scraping the topics details like topic names, topic urls

In [13]:
outer_div_tags=(contents.find_all('div',{'class':'_2jmnnhuP2AL87nZM3exPTR'}))

In [14]:
inner_div_tags=get_inner_div_tags(outer_div_tags)

In [15]:
total_a_tags=get_a_tags(inner_div_tags)

In [16]:
topic_names,urls=get_topic_details(total_a_tags)

### Getting the topic info

In [21]:
topic_description,topic_members,topic_online,topic_createddate=get_topic_infos(urls)

### Creating a dataframe

In [22]:
data={'Topic Names':topic_names,
      'Topic Urls':urls,
      'Topic Description':topic_description,
      'Followers':topic_members,
      'Active':topic_online,
      'Created Date':topic_createddate}

In [23]:
df=pd.DataFrame(data)

In [24]:
df

,Topic Names,Topic Urls,Topic Description,Followers,Active,Created Date
0,AskReddit,https://www.reddit.com/r/AskReddit/,r/AskReddit is the place to ask and answer tho...,34.5m,70.6k,"Jan 25, 2008"
1,NoStupidQuestions,https://www.reddit.com/r/NoStupidQuestions/,Ask away!,2.5m,14.2k,"Feb 2, 2013"
2,DestinyTheGame,https://www.reddit.com/r/DestinyTheGame/,,,,
3,explainlikeimfive,https://www.reddit.com/r/explainlikeimfive/,,,,
4,AskMen,https://www.reddit.com/r/AskMen/,It's time to stop.,3.2m,9.9k,"Aug 30, 2010"
...,...,...,...,...,...,...
554,Youniqueamua,https://www.reddit.com/r/Youniqueamua/,"The horrors of multi-level marketing ""makeup"" ...",140k,27,"Jul 27, 2017"
555,sneakermarket,https://www.reddit.com/r/sneakermarket/,Confused on how to buy/sell/trade your sneaker...,166k,203,"Aug 26, 2011"
556,SarahSnark,https://www.reddit.com/r/SarahSnark/,,,,
557,RepLadiesBST,https://www.reddit.com/r/RepLadiesBST/,"A marketplace dedicated to selling, purchasing...",51.5k,225,"Feb 21, 2018"


### Saving the data into a CSV

In [25]:
df.to_csv('PagesDetailsOnReddit.csv',index=None)

In [26]:
driver.quit()